In [3]:
from fredapi import Fred
import pandas as pd
import numpy as np

API_KEY = "2ccf5b794d310f8cde1d30c463f8d2d4"
fred = Fred(api_key=API_KEY)

# --- Define Series for Howard County ---
howard_series = {
    "Median_Listing_Price": ("MEDLISPRI24027", "M"),
    "Active_Listings": ("ACTLISCOU24027", "M"),
    "Unemployed_Persons": ("LAUCN240270000000004", "M"),
    "Unemployment_Rate": ("MDHOWA0URN", "M"),
    "Civilian_Labor_Force": ("MDHOWA0LFN", "M"),
    "Population_Annual": ("MDHOWA0POP", "A"),
    "Permits_Annual": ("BPPRIV024027", "A"),
    "HPI_AllTransactions": ("ATNHPIUS24027A", "A"),
    "Real_GDP": ("REALGDPALL24027", "A"),
    "Poverty_All_Ages": ("PPAAMD24027A156NCEN", "A")
}

# --- Pull Each Series ---
frames = []
for col_name, (series_id, freq) in howard_series.items():
    data = fred.get_series(series_id)

    # Skip empty or invalid series
    if data.empty:
        print(f"⚠️ Warning: No data returned for {series_id} ({col_name})")
        continue

    # Ensure datetime index
    df = pd.DataFrame({"Date": pd.to_datetime(data.index, errors="coerce"), col_name: data.values})
    df = df.dropna(subset=["Date"])  # remove rows with invalid dates

    if freq == "M":
        df["Date"] = df["Date"].dt.to_period("M").dt.to_timestamp("M")  # month-end
    else:
        df["Date"] = df["Date"].dt.to_period("Y").dt.to_timestamp("M")  # year-end
        df = df.set_index("Date").resample("M").ffill().reset_index()  # upsample annual -> monthly
    
    frames.append(df)

# --- Merge All Indicators on Date ---
from functools import reduce
howard_df = reduce(lambda left, right: pd.merge(left, right, on="Date", how="outer"), frames)
howard_df = howard_df.sort_values("Date")
howard_df.head()

C:\Users\Thanos\AppData\Local\Temp\ipykernel_9644\2047967946.py:40: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.set_index("Date").resample("M").ffill().reset_index()  # upsample annual -> monthly
C:\Users\Thanos\AppData\Local\Temp\ipykernel_9644\2047967946.py:40: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.set_index("Date").resample("M").ffill().reset_index()  # upsample annual -> monthly
C:\Users\Thanos\AppData\Local\Temp\ipykernel_9644\2047967946.py:40: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.set_index("Date").resample("M").ffill().reset_index()  # upsample annual -> monthly
C:\Users\Thanos\AppData\Local\Temp\ipykernel_9644\2047967946.py:40: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df = df.set_index("Date").resample("M").ffill().

,Date,Median_Listing_Price,Active_Listings,Unemployed_Persons,Unemployment_Rate,Civilian_Labor_Force,Population_Annual,Permits_Annual,HPI_AllTransactions,Real_GDP,Poverty_All_Ages
0,1970-01-31,NaN,NaN,NaN,NaN,NaN,62.394,NaN,NaN,NaN,NaN
1,1970-02-28,NaN,NaN,NaN,NaN,NaN,62.394,NaN,NaN,NaN,NaN
2,1970-03-31,NaN,NaN,NaN,NaN,NaN,62.394,NaN,NaN,NaN,NaN
3,1970-04-30,NaN,NaN,NaN,NaN,NaN,62.394,NaN,NaN,NaN,NaN
4,1970-05-31,NaN,NaN,NaN,NaN,NaN,62.394,NaN,NaN,NaN,NaN


In [4]:
howard_df.to_csv("Howard_County_Indicators.csv", index=False)
print("✅ Howard County dataset exported!")
howard_df.tail()

✅ Howard County dataset exported!


,Date,Median_Listing_Price,Active_Listings,Unemployed_Persons,Unemployment_Rate,Civilian_Labor_Force,Population_Annual,Permits_Annual,HPI_AllTransactions,Real_GDP,Poverty_All_Ages
665,2025-06-30,692450.0,449.0,6011.0,3.3,184190.0,NaN,NaN,NaN,NaN,NaN
666,2025-07-31,680000.0,478.0,6266.0,3.4,185196.0,NaN,NaN,NaN,NaN,NaN
667,2025-08-31,669950.0,511.0,6888.0,3.8,183189.0,NaN,NaN,NaN,NaN,NaN
668,2025-09-30,649450.0,503.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,2025-10-31,612450.0,497.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
howard_df.notna().sum()

Date                    670
Median_Listing_Price    112
Active_Listings         112
Unemployed_Persons      428
Unemployment_Rate       428
Civilian_Labor_Force    428
Population_Annual       649
Permits_Annual          409
HPI_AllTransactions     589
Real_GDP                265
Poverty_All_Ages        349
dtype: int64